# 1. ライブラリのインポート

[PytorchのMNISTのサンプル](https://github.com/pytorch/examples/blob/master/mnist/main.py)をコピーしただけなのでいらないものもあるかもしれません。

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
from __future__ import print_function
import argparse
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# 2. モデルの定義

サンプルのまんまです。

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


# 3. 学習データの準備

サンプルのまんまです。

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
        transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64)

# 4. seedの設定

サンプルのまんまです。

In [ ]:
torch.manual_seed(1)

# 5. 学習を実行する関数

サンプルをもとに記載しています。


In [ ]:
def train(mode):
    device = torch.device(mode)
    
    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=1.0)
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if mode == "cuda" else {}
    
    model.train()

    start = time.time()
    
    for epoch in range(1, 15):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    end = time.time()
    
    processing_time = end - start
    print("Pprocessing Time(sec): " + str(processing_time))

# 6. 学習を実行

## 6-1. GPUの場合

GPU利用の場合です。

In [ ]:
train("cuda")

## 6-2. CPUの場合

CPUのみ利用の場合です。

In [ ]:
train("cpu")